In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
classes = ["Balcony", "Bar", "Bathroom", "Bedroom", "Business Centre", "Dining room", "Exterior",
           "Gym", "Living room", "Lobby", "Patio", "Pool", "Restaurant", "Sauna", "Spa"]


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
num_epochs = 20
batch_size = 32
learning_rate = 0.001


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet statistics
])

In [6]:
dataset = ImageFolder("../input/hotel-images/Dataset/", transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

print("Train Size : " + str(train_size));
print("Test Size : " + str(test_size))

Train Size : 59196
Test Size : 14799


In [7]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


In [10]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.features = torchvision.models.vgg16(pretrained=True)
        num_features = self.features.classifier[6].in_features
        self.features.classifier[6] = nn.Linear(num_features, len(classes))
    
    def forward(self, x):
        x = self.features(x)
        return x

model = CNNModel().to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [12]:
total_step = len(train_loader)
print("Total Steps : " + str(total_step))

Total Steps : 1850


In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_correct = 0
    total_samples = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Print training progress
        if (i + 1) == total_step:
            print(f"Epoch [{epoch + 1}/{num_epochs}]"
                  f"Train Loss: {loss.item():.4f}, Train Accuracy: {(total_correct / total_samples) * 100:.2f}%")
            
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0

                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
        
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
        
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
    
                print(f"Test Accuracy: {(100 * correct / total):.2f}%")
                print("")
        
        if epoch == num_epochs - 1:
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                all_predicted = []
                all_labels = []

                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
    
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
    
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    # Collect predicted and true labels for classification report
                    all_predicted.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())

            avg_precision = precision_score(all_labels, all_predicted, average='macro')
            avg_recall = recall_score(all_labels, all_predicted, average='macro')
            avg_f1_score = f1_score(all_labels, all_predicted, average='macro')

            print(f"Average Precision: {avg_precision:.4f}")
            print(f"Average Recall: {avg_recall:.4f}")
            print(f"Average F1-Score: {avg_f1_score:.4f}")
            print("")

Epoch [1/20] Train Loss: 0.6443, Train Accuracy: 64.56%
Test Accuracy: 58.62%

Epoch [2/20] Train Loss: 0.4065, Train Accuracy: 69.47%
Test Accuracy: 62.96%

Epoch [3/20] Train Loss: 0.3344, Train Accuracy: 74.11%
Test Accuracy: 66.31%

Epoch [4/20] Train Loss: 0.2255, Train Accuracy: 78.61%
Test Accuracy: 70.92%

Epoch [5/20] Train Loss: 0.1989, Train Accuracy: 82.51%
Test Accuracy: 75.52%

Epoch [6/20] Train Loss: 0.1615, Train Accuracy: 86.63%
Test Accuracy: 79.87%

Epoch [7/20] Train Loss: 0.1219, Train Accuracy: 89.84%
Test Accuracy: 84.82%

Epoch [8/20] Train Loss: 0.0931, Train Accuracy: 91.52%
Test Accuracy: 89.62%

Epoch [9/20] Train Loss: 0.0781, Train Accuracy: 92.54%
Test Accuracy: 91.32%

Epoch [10/20] Train Loss: 0.0794, Train Accuracy: 92.98%
Test Accuracy: 92.56%

Epoch [11/20] Train Loss: 0.0744, Train Accuracy: 93.76%
Test Accuracy: 92.09%

Epoch [12/20] Train Loss: 0.0763, Train Accuracy: 93.33%
Test Accuracy: 92.32%

Epoch [13/20] Train Loss: 0.0711, Train Accuracy:

In [18]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"Test Accuracy: {(100 * correct / total):.2f}%")

Validation Accuracy: 92.91%


In [19]:
model_path = "vgg16.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved at: {model_path}")

Model saved at: vgg16.pth
